Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [37]:
def compute_loss(logits, labels, weights, loss_l2_constant):
    loss_tensor = tf.nn.softmax_cross_entropy_with_logits(logits, labels)
    loss_tensor_l2_inp_hidden = loss_l2_constant * tf.nn.l2_loss(weights['inp_hidden'])
    loss_tensor_l2_hidden_out = loss_l2_constant * tf.nn.l2_loss(weights['hidden_out'])
    loss = tf.reduce_mean(
      loss_tensor + loss_tensor_l2_inp_hidden + loss_tensor_l2_hidden_out
    )
    return loss

In [50]:
batch_size = 128
hidden_layer_size=1024
l2_constant=0.001

def compute_logits(dataset, weights, biases):
  inp_hidden = tf.matmul(dataset, weights['inp_hidden']) + biases['inp_hidden']
  hidden_relu = tf.nn.relu(inp_hidden)
  hidden_out = tf.matmul(hidden_relu, weights['hidden_out']) + biases['hidden_out']
  return hidden_out

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
    'inp_hidden' : tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size])),
    'hidden_out' : tf.Variable(tf.truncated_normal([hidden_layer_size,num_labels]))
  }

  biases = {
    'inp_hidden' : tf.Variable(tf.zeros([hidden_layer_size])),
    'hidden_out' : tf.Variable(tf.zeros([num_labels]))
  }
  
  # Training computation.
  # Compute the inpu-hidden layer
  train_logits = compute_logits(tf_train_dataset, weights, biases)

  loss = compute_loss(train_logits, tf_train_labels, weights, l2_constant)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_logits)
  valid_logits = compute_logits(tf_valid_dataset, weights, biases)
  valid_prediction = tf.nn.softmax(valid_logits)
  test_logits = compute_logits(tf_test_dataset, weights, biases)
  test_prediction = tf.nn.softmax(test_logits)

In [51]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 623.679321
Minibatch accuracy: 13.3%
Validation accuracy: 34.9%
Minibatch loss at step 500: 195.864990
Minibatch accuracy: 84.4%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 116.546677
Minibatch accuracy: 77.3%
Validation accuracy: 81.1%
Minibatch loss at step 1500: 68.570114
Minibatch accuracy: 89.8%
Validation accuracy: 82.5%
Minibatch loss at step 2000: 41.218376
Minibatch accuracy: 88.3%
Validation accuracy: 84.4%
Minibatch loss at step 2500: 25.071230
Minibatch accuracy: 92.2%
Validation accuracy: 85.6%
Minibatch loss at step 3000: 15.445635
Minibatch accuracy: 84.4%
Validation accuracy: 86.3%
Test accuracy: 93.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [54]:
batch_size = 8
hidden_layer_size=1024
l2_constant=0.001

def compute_logits(dataset, weights, biases):
  inp_hidden = tf.matmul(dataset, weights['inp_hidden']) + biases['inp_hidden']
  hidden_relu = tf.nn.relu(inp_hidden)
  hidden_out = tf.matmul(hidden_relu, weights['hidden_out']) + biases['hidden_out']
  return hidden_out

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
    'inp_hidden' : tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size])),
    'hidden_out' : tf.Variable(tf.truncated_normal([hidden_layer_size,num_labels]))
  }

  biases = {
    'inp_hidden' : tf.Variable(tf.zeros([hidden_layer_size])),
    'hidden_out' : tf.Variable(tf.zeros([num_labels]))
  }
  
  # Training computation.
  # Compute the inpu-hidden layer
  train_logits = compute_logits(tf_train_dataset, weights, biases)

  loss = compute_loss(train_logits, tf_train_labels, weights, l2_constant)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_logits)
  valid_logits = compute_logits(tf_valid_dataset, weights, biases)
  valid_prediction = tf.nn.softmax(valid_logits)
  test_logits = compute_logits(tf_test_dataset, weights, biases)
  test_prediction = tf.nn.softmax(test_logits)

In [55]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 680.513550
Minibatch accuracy: 0.0%
Validation accuracy: 13.8%
Minibatch loss at step 500: 102575520.000000
Minibatch accuracy: 25.0%
Validation accuracy: 31.1%
Minibatch loss at step 1000: 1499246362624.000000
Minibatch accuracy: 25.0%
Validation accuracy: 38.6%
Minibatch loss at step 1500: 53786631362052096.000000
Minibatch accuracy: 37.5%
Validation accuracy: 39.7%
Minibatch loss at step 2000: 708635701498469154816.000000
Minibatch accuracy: 50.0%
Validation accuracy: 41.7%
Minibatch loss at step 2500: 35173578293590682622754816.000000
Minibatch accuracy: 25.0%
Validation accuracy: 29.3%
Minibatch loss at step 3000: 18441736004054468357250875392.000000
Minibatch accuracy: 62.5%
Validation accuracy: 34.4%
Test accuracy: 37.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [102]:
def compute_loss(logits, labels, weights, loss_l2_constant, num_hidden=1):
    loss_tensor = tf.nn.softmax_cross_entropy_with_logits(logits, labels)
    loss_tensors = loss_tensor
    for i in range(0,num_hidden):
            loss_tensor_l2_inp_hidden = loss_l2_constant * tf.nn.l2_loss(weights['inp_hidden' + str(i+1)])
            loss_tensors += loss_tensor_l2_inp_hidden
    loss_tensor_l2_hidden_out = loss_l2_constant * tf.nn.l2_loss(weights['hidden_out'])
    loss_tensors += loss_tensor_l2_hidden_out
    loss = tf.reduce_mean(loss_tensors)
    return loss

In [103]:
def compute_logits(dataset, weights, biases, num_hidden=1):
    inp_hidden = dataset
    for i in range(0,num_hidden):
        inp_hidden = tf.matmul(inp_hidden, weights['inp_hidden' + str(i+1)]) + biases['inp_hidden' + str(i+1)]
        hidden_relu = tf.nn.relu(inp_hidden)
    hidden_out = tf.matmul(hidden_relu, weights['hidden_out']) + biases['hidden_out']
    return hidden_out

In [106]:
batch_size = 128
hidden_layer_size=1024
l2_constant=0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
    'inp_hidden1' : tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size])),
    'inp_hidden2' : tf.Variable(tf.truncated_normal([hidden_layer_size, hidden_layer_size])),
    'hidden_out' : tf.Variable(tf.truncated_normal([hidden_layer_size,num_labels]))
  }

  biases = {
    'inp_hidden1' : tf.Variable(tf.zeros([hidden_layer_size])),
    'inp_hidden2' : tf.Variable(tf.zeros([hidden_layer_size])),
    'hidden_out' : tf.Variable(tf.zeros([num_labels]))
  }
  
  # Training computation.
  # Compute the inpu-hidden layer
  train_logits = compute_logits(tf_train_dataset, weights, biases,num_hidden=1)

  loss = compute_loss(train_logits, tf_train_labels, weights, l2_constant,num_hidden=1)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_logits)
  valid_logits = compute_logits(tf_valid_dataset, weights, biases,num_hidden=1)
  valid_prediction = tf.nn.softmax(valid_logits)
  test_logits = compute_logits(tf_test_dataset, weights, biases,num_hidden=1)
  test_prediction = tf.nn.softmax(test_logits)

In [107]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 606.640747
Minibatch accuracy: 10.2%
Validation accuracy: 30.7%
Minibatch loss at step 500: 198.435394
Minibatch accuracy: 77.3%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 115.074440
Minibatch accuracy: 82.8%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 68.531403
Minibatch accuracy: 90.6%
Validation accuracy: 82.8%
Minibatch loss at step 2000: 41.142849
Minibatch accuracy: 91.4%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.132908
Minibatch accuracy: 90.6%
Validation accuracy: 85.6%
Minibatch loss at step 3000: 15.405401
Minibatch accuracy: 90.6%
Validation accuracy: 86.2%
Test accuracy: 92.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [17]:
def compute_loss(logits, labels, weights, loss_l2_constant):
    num_hidden=3
    loss_tensor = tf.nn.softmax_cross_entropy_with_logits(logits, labels)
    loss_tensors = loss_tensor
    for i in range(0,num_hidden):
        loss_tensor_l2_inp_hidden = loss_l2_constant * tf.nn.l2_loss(weights['h' + str(i+1)])
        loss_tensors += loss_tensor_l2_inp_hidden
    loss_tensor_l2_hidden_out = loss_l2_constant * tf.nn.l2_loss(weights['out'])
    loss_tensors += loss_tensor_l2_hidden_out
    loss = tf.reduce_mean(loss_tensors)
    return loss

def compute_logits(dataset, weights, biases, use_dropout):
    num_hidden=3
    input_layer = dataset
    for i in range(0,num_hidden):
        hidden_layer = tf.add(tf.matmul(input_layer, weights['h' + str(i+1)]), biases['h' + str(i+1)])
        if use_dropout:
            hidden_layer = tf.nn.dropout(hidden_layer, 0.5)
        else:
            hidden_layer = hidden_layer * 0.5
        hidden_layer = tf.nn.relu(hidden_layer)
        input_layer = hidden_layer
    output_layer = tf.add(tf.matmul(input_layer, weights['out']), biases['out'])
    return output_layer

batch_size=2048
hidden_layer_1_size=4096
hidden_layer_2_size=2048
hidden_layer_3_size=1024
l2_constant=0.001
image_size=28
num_labels=10

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
    'h1' : tf.get_variable(name="weights_h1", shape=(image_size*image_size,hidden_layer_1_size),initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32),
    'h2' : tf.get_variable(name="weights_h2", shape=(hidden_layer_1_size,hidden_layer_2_size),initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32),
    'h3' : tf.get_variable(name="weights_h3", shape=(hidden_layer_2_size,hidden_layer_3_size),initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32),
    'out' : tf.get_variable(name="weights_out", shape=(hidden_layer_3_size,num_labels),initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
  }

  biases = {
    'h1' : tf.Variable(tf.zeros([hidden_layer_1_size])),
    'h2' : tf.Variable(tf.zeros([hidden_layer_2_size])),
    'h3' : tf.Variable(tf.zeros([hidden_layer_3_size])),
    'out' : tf.Variable(tf.zeros([num_labels]))
  }
    
  global_step = tf.Variable(0)
  
  # Training computation.
  # Compute the inpu-hidden layer
  train_logits = compute_logits(tf_train_dataset, weights, biases, True)

  loss = compute_loss(train_logits, tf_train_labels, weights, l2_constant)

  learning_rate = tf.train.exponential_decay(0.0001, global_step, 1000, 0.96)
  
  # Optimizer.
  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_logits)
  valid_logits = compute_logits(tf_valid_dataset, weights, biases, False)
  valid_prediction = tf.nn.softmax(valid_logits)
  test_logits = compute_logits(tf_test_dataset, weights, biases, False)
  test_prediction = tf.nn.softmax(test_logits)


In [18]:
num_steps = 10001
print('Num Steps:', num_steps)
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Num Steps: 10001
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 5.131063
Minibatch accuracy: 8.5%
Validation accuracy: 44.3%
Minibatch loss at step 100: 3.051163
Minibatch accuracy: 81.5%
Validation accuracy: 84.0%
Minibatch loss at step 200: 2.625056
Minibatch accuracy: 85.6%
Validation accuracy: 85.3%
Minibatch loss at step 300: 2.282309
Minibatch accuracy: 86.1%
Validation accuracy: 86.3%
Minibatch loss at step 400: 2.007503
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 500: 1.729271
Minibatch accuracy: 89.4%
Validation accuracy: 87.8%
Minibatch loss at step 600: 1.555037
Minibatch accuracy: 88.9%
Validation accuracy: 88.2%
Minibatch loss at step 700: 1.377724
Minibatch accuracy: 89.3%
Validation accuracy: 88.5%
Minibatch loss at step 800: 1.258736
Minibatch accuracy: 88.9%
Validation accuracy: 88.9%
Minibatch loss at step 900: 1.094962
Minibatch accuracy: 89.8%
Validation accuracy: 89.0%


In [11]:
print(train_labels.shape[0] / batch_size)

1562


In [12]:
print(train_labels.shape)

(200000, 10)


In [16]:
# With Batch Normalization

In [20]:
def compute_loss(logits, labels, weights, loss_l2_constant):
    num_hidden=3
    loss_tensor = tf.nn.softmax_cross_entropy_with_logits(logits, labels)
    loss_tensors = loss_tensor
    for i in range(0,num_hidden):
        loss_tensor_l2_inp_hidden = loss_l2_constant * tf.nn.l2_loss(weights['h' + str(i+1)])
        loss_tensors += loss_tensor_l2_inp_hidden
    loss_tensor_l2_hidden_out = loss_l2_constant * tf.nn.l2_loss(weights['out'])
    loss_tensors += loss_tensor_l2_hidden_out
    loss = tf.reduce_mean(loss_tensors)
    return loss

def compute_logits(dataset, weights, biases, use_dropout, use_batchnorm):
    num_hidden=3
    input_layer = dataset
    for i in range(0,num_hidden):
        hidden_layer = tf.add(tf.matmul(input_layer, weights['h' + str(i+1)]), biases['h' + str(i+1)])
        if use_dropout:
            hidden_layer = tf.nn.dropout(hidden_layer, 0.5)
        else:
            hidden_layer = hidden_layer * 0.5
        if use_batchnorm:
            mean, variance = tf.nn.moments(dataset, [0])
        hidden_layer = tf.nn.relu(hidden_layer)
        input_layer = hidden_layer
    output_layer = tf.add(tf.matmul(input_layer, weights['out']), biases['out'])
    return output_layer

batch_size=128
hidden_layer_1_size=4096
hidden_layer_2_size=2048
hidden_layer_3_size=1024
l2_constant=0.001
image_size=28
num_labels=10

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
    'h1' : tf.get_variable(name="weights_h1", shape=(image_size*image_size,hidden_layer_1_size),initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32),
    'h2' : tf.get_variable(name="weights_h2", shape=(hidden_layer_1_size,hidden_layer_2_size),initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32),
    'h3' : tf.get_variable(name="weights_h3", shape=(hidden_layer_2_size,hidden_layer_3_size),initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32),
    'out' : tf.get_variable(name="weights_out", shape=(hidden_layer_3_size,num_labels),initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
  }

  biases = {
    'h1' : tf.Variable(tf.zeros([hidden_layer_1_size])),
    'h2' : tf.Variable(tf.zeros([hidden_layer_2_size])),
    'h3' : tf.Variable(tf.zeros([hidden_layer_3_size])),
    'out' : tf.Variable(tf.zeros([num_labels]))
  }
    
  global_step = tf.Variable(0)
  
  # Training computation.
  # Compute the inpu-hidden layer
  train_logits = compute_logits(tf_train_dataset, weights, biases, True, True)

  loss = compute_loss(train_logits, tf_train_labels, weights, l2_constant)

  learning_rate = tf.train.exponential_decay(0.0001, global_step, 1000, 0.96)
  
  # Optimizer.
  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_logits)
  valid_logits = compute_logits(tf_valid_dataset, weights, biases, False, True)
  valid_prediction = tf.nn.softmax(valid_logits)
  test_logits = compute_logits(tf_test_dataset, weights, biases, False, True)
  test_prediction = tf.nn.softmax(test_logits)

In [ ]:
num_steps = 3001
print('Num Steps:', num_steps)
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))